In [4]:
function mit18086_multigrid
%MIT18086_MULTIGRID
%    Sets up a 1d Poisson test problem and solves it by multigrid.
%    Method uses twogrid recursively using Gauss-Seidel for smoothing
%    and elimination to solve at coarsest level (n<4).
%    Number of pre- and postsmoothing and coarse grid iteration steps
%    can be prescribed.

% 04/2007 by Benjamin Seibold
%            http://www-math.mit.edu/~seibold/
% Feel free to modify for teaching and learning.
%---------------------------------------------------------------------
levels = 5;                                          % size of problem
nu1 = 2;                           % number of presmoothing iterations
nu2 = 2;                          % number of postsmoothing iterations
gamma = 2;   % number of coarse grid iterations (1=V-cycle, 2=W-cycle)
%---------------------------------------------------------------------
n = 2^(levels+2)-1;                            % number of grid points
h = 1/(n+1);
x = (h:h:(1-h))';
f = pi^2*(sin(pi*x)+4^2*sin(pi*4*x)+9^2*sin(pi*9*x));
A = spdiags(ones(n,1)*[-1 2 -1],-1:1,n,n);
b = f*h^2;
uc = A\b;

global t level, t = 0; level = levels;
clf, subplot(2,2,3:4), hold on
u = twogrid(A,b,nu1,nu2,gamma);
hold off, axis tight
subplot(2,2,1), plot(x,u,'b.-',x,uc,'r.--')
title('correct solution and multigrid approximation')
subplot(2,2,2), plot(x,uc-u,'r.-')
title('error')

%=====================================================================

function x = twogrid(A,b,nu1,nu2,gamma,x0)
%TWOGRID
%    Recursive twogrid cycle for 1d Poisson problem.
%    nu1 = number of presmoothing iterations (Gauss-Seidel)
%    nu2 = number of postsmoothing iterations (Gauss-Seidel)
%    gamma = number of coarse grid iterations (1=V-cycle, 2=W-cycle)
%    x0 = starting vector (0 if not prescribed)
global t level
n = length(b);
if n<4
   x = A\b;                          % solve exactly at coarsest level
else
   G = speye(n)-tril(A)\A; cG = tril(A)\b;       % create Gauss-Seidel
   I = spdiags(ones(n-2,1)*[1 2 1],-2:0,n,n-2); % create interpolation
   I = I(:,1:2:end)/2; R = I'/2;            % and restriction matrices
   if nargin<6, x = b*0; else, x = x0; end           % starting vector
   for i = 1:nu1, x = G*x+cG; end                       % presmoothing
   r = b-A*x;                                       % compute residual
   rh = R*r;                        % restrict residual to coarse grid
   t = t+1; level = level-1; plot([t-1 t], [level+1 level],'bo-')
   eh = rh*0;                                        % starting vector
   for i = 1:gamma
      eh = twogrid(R*A*I,rh,nu1,nu2,gamma,eh); % coarse grid iteration
   end
   e = I*eh;                                       % interpolate error
   t = t+1; level = level+1; plot([t-1 t],[level-1 level],'bo-')
   x = x+e;                                          % update solution
   for i = 1:nu2, x = G*x+cG; end                      % postsmoothing
end


SyntaxError: invalid syntax (<ipython-input-4-2db8c3534ea2>, line 1)

In [5]:
import numpy as np
import warnings
from scipy import sparse
from scipy.sparse import linalg

In [6]:
# def twogrid(A, b, nu1, nu2, gamma, x0=None):
#     #     TWOGRID
#     #         Recursive twogrid cycle for 1d Poisson problem.
#     #         nu1 = number of presmoothing iterations (Gauss-Seidel)
#     #         nu2 = number of postsmoothing iterations (Gauss-Seidel)
#     #         gamma = number of coarse grid iterations (1=V-cycle, 2=W-cycle)
#     #         x0 = starting vector (0 if not prescribed)
#     warnings.filterwarnings("ignore")
#     n = b.size
#     if n < 4:
#         x = linalg.spsolve(A, b)
#     else:
#         G = sparse.eye(n) - linalg.inv(sparse.tril(A)).dot(A)
#         cG = linalg.inv(sparse.tril(A)).dot(b)
#         I = sparse.diags([np.full((n - 2), 0.5), np.ones((n - 2)), 
#                         np.full((n - 2), 0.5)], [0, -1, -2], 
#                         shape=(n, n - 2), format='csr')
#         I = I[:, ::2]
#         R = I.T / 2
#         if x0 is None:
#             x = np.zeros_like(b)
#         else:
#             x = x0.copy()
#         for i in range(nu1):
#             x = G.dot(x) + cG
#         r = b - A.dot(x)
#         rh = R.dot(r)
#         eh = np.zeros_like(rh)
#         for i in range(gamma):
#             eh = twogrid(R.dot(A.dot(I)), rh, nu1, nu2, gamma, eh)
#         e = I.dot(eh)
#         x += e
#         for i in range(nu2):
#             x = G.dot(x) + cG
#     return x

In [7]:
def twogrid_with_fixes(A, b, nu1, nu2, gamma, x0=None, damp=2./3, galerkin=True):
    #     TWOGRID
    #         Recursive twogrid cycle for 1d Poisson problem.
    #         nu1 = number of presmoothing iterations (Gauss-Seidel)
    #         nu2 = number of postsmoothing iterations (Gauss-Seidel)
    #         gamma = number of coarse grid iterations (1=V-cycle, 2=W-cycle)
    #         x0 = starting vector (0 if not prescribed)
    warnings.filterwarnings("ignore")
    n = b.size
    if n < 4:
        x = linalg.spsolve(A, b)
    else:
        I = sparse.diags([np.full((n - 2), 0.5), np.ones((n - 2)), 
                        np.full((n - 2), 0.5)], [0, -1, -2], 
                        shape=(n, n - 2), format='csr')
        I = I[:, ::2]
        R = I.T / 2
        if x0 is None:
            x = np.zeros_like(b)
        else:
            x = x0.copy()
        for i in range(nu1):
            x = damp * 0.5 * (b - A.dot(x) + 2 * x) + (1 - damp) * x
        r = b - A.dot(x)
        rh = R.dot(r)
        eh = np.zeros_like(rh)
        for i in range(gamma):
            if galerkin:
                new_A = R.dot(A.dot(I))
            else:
                new_A = A[:(n+1) / 2 - 1, :(n+1) / 2 - 1]
            eh = twogrid_with_fixes(new_A, rh, nu1, nu2, gamma, eh, galerkin=galerkin)
        e = I.dot(eh)
        x += e
        for i in range(nu2):
            x = damp * 0.5 * (b - A.dot(x) + 2 * x) + (1 - damp) * x
    return x

In [20]:
levels = 7                  # size of problem
nu1 = 2                     # number of presmoothing iterations
nu2 = 2                     # number of postsmoothing iterations
gamma = 1                   # number of coarse grid iterations (1=V-cycle, 2=W-cycle)
n = 2 ** (levels + 2) - 1   # number of grid points
h = 1.0 / (n + 1)
x = np.linspace(h, 1-h, n)
f = np.ones((n)) #np.pi ** 2 * (np.sin(np.pi * x) + 4 ** 2 * np.sin(np.pi * 4 * x) + 
                 #9 ** 2 * np.sin(np.pi * 9 * x))
A = sparse.diags([np.full((n), 2.), -np.ones((n - 1)), 
              -np.ones((n - 1))], [0, -1, 1], format='csr')
b = f
uc = linalg.spsolve(A, b)
u = np.zeros_like(b)

In [9]:
n

511

Без Галёркина:

In [14]:
res_arr = []
for i in range(50):
    u = twogrid_with_fixes(A, b, nu1, nu2, gamma, u, galerkin=False)
    res_arr.append(np.linalg.norm(A.dot(u) - b) / np.linalg.norm(b))
res_arr = np.array(res_arr)
res_arr

array([ 1.3460233 ,  1.31034918,  1.27716884,  1.24616516,  1.21708481,
        1.18971595,  1.16387841,  1.13941754,  1.11619973,  1.09410893,
        1.07304382,  1.05291557,  1.03364591,  1.01516564,  0.99741328,
        0.98033407,  0.96387901,  0.94800414,  0.93266987,  0.91784047,
        0.90348358,  0.8895698 ,  0.87607241,  0.862967  ,  0.85023125,
        0.83784472,  0.82578861,  0.81404566,  0.80259993,  0.7914367 ,
        0.78054238,  0.76990436,  0.75951092,  0.74935121,  0.7394151 ,
        0.72969317,  0.7201766 ,  0.71085718,  0.70172721,  0.69277948,
        0.68400724,  0.67540414,  0.66696421,  0.65868183,  0.65055172,
        0.64256888,  0.63472861,  0.62702645,  0.61945819,  0.61201984])

In [15]:
res_arr[1:] / res_arr[:-1]

array([ 0.97349665,  0.97467825,  0.97572468,  0.97666413,  0.97751278,
        0.9782826 ,  0.97898331,  0.97962309,  0.98020891,  0.98074679,
        0.98124191,  0.98169876,  0.98212127,  0.98251285,  0.9828765 ,
        0.98321485,  0.98353022,  0.98382468,  0.98410005,  0.98435796,
        0.98459986,  0.98482706,  0.98504072,  0.9852419 ,  0.98543157,
        0.98561057,  0.98577971,  0.98593969,  0.98609117,  0.98623475,
        0.98637098,  0.98650036,  0.98662335,  0.98674038,  0.98685185,
        0.98695812,  0.98705953,  0.98715639,  0.987249  ,  0.98733762,
        0.9874225 ,  0.98750388,  0.98758198,  0.987657  ,  0.98772913,
        0.98779855,  0.98786543,  0.98792992,  0.98799216])

С Галёркиным:

In [21]:
res_arr = []
for i in range(50):
    u = twogrid_with_fixes(A, b, nu1, nu2, gamma, u, galerkin=True)
    res_arr.append(np.linalg.norm(A.dot(u) - b) / np.linalg.norm(b))
res_arr = np.array(res_arr)
res_arr

array([ 3.69273461,  2.617871  ,  2.21574204,  1.98169709,  1.82055647,
        1.6992759 ,  1.60289609,  1.52339053,  1.45596663,  1.39755748,
        1.34610056,  1.30015306,  1.25867145,  1.22087877,  1.18618174,
        1.15411721,  1.12431672,  1.09648231,  1.07036961,  1.0457758 ,
        1.02253078,  1.00049062,  0.97953257,  0.95955119,  0.9404554 ,
        0.92216604,  0.90461398,  0.88773858,  0.87148639,  0.85581014,
        0.84066786,  0.82602212,  0.81183951,  0.79809001,  0.78474665,
        0.77178509,  0.75918329,  0.74692129,  0.73498091,  0.72334559,
        0.71200019,  0.70093084,  0.69012479,  0.67957032,  0.66925663,
        0.65917372,  0.64931232,  0.63966384,  0.63022028,  0.62097419])

In [22]:
res_arr[1:] / res_arr[:-1]

array([ 0.70892476,  0.84639084,  0.89437175,  0.91868555,  0.93338269,
        0.94328184,  0.9503988 ,  0.9557409 ,  0.9598829 ,  0.96318082,
        0.96586622,  0.96809483,  0.96997415,  0.97158028,  0.97296828,
        0.97417898,  0.97524327,  0.97618502,  0.97702307,  0.97777247,
        0.97844548,  0.97905222,  0.97960111,  0.98009925,  0.98055265,
        0.98096649,  0.98134519,  0.9816926 ,  0.98201206,  0.98230649,
        0.98257846,  0.98283022,  0.98306377,  0.98328088,  0.98348312,
        0.98367189,  0.98384843,  0.98401387,  0.98416922,  0.98431538,
        0.98445316,  0.98458329,  0.98470644,  0.98482322,  0.98493416,
        0.98503976,  0.98514046,  0.98523669,  0.9853288 ])